In [1]:
import os
import pandas as pd
from collections import Counter
import numpy as np
import plotly.graph_objects as go


from func.graph import graph
from func.cross_points_f import cross_points_f
from func.cross_points_mx_w import cross_points_mx_w

# Подготовка

In [2]:
file = (os.path.abspath('BPLA, series 1 - инт.csv')).replace('\\', '/')
df = pd.read_csv(file, sep = ',')
df = pd.DataFrame(df.iloc[:,1:])

In [3]:
mx = np.array(df.loc[:, 'mx, Н*м'].values)
fx = np.array(df.loc[:, 'Fx, Н'].values)
w = np.array(df.loc[:, 'Wв, Вт'].values)
p = np.array(df.loc[:, 'P, Н/Вт'].values)

### Определим размер матрицы для графика

In [4]:
                                    # Используется при не одинаковом числе расчётов
for number, n_number in Counter(df['a, град']).items():
    break
for number, a_number in Counter(df['n, об/мин']).items():
    break
print(a_number, n_number)

66 11


### Подготовим матрицы для графиков 

In [5]:
k = 0

mx_m, fx_m, w_m, p_m = np.zeros((a_number, n_number)), np.zeros((a_number, n_number)), np.zeros((a_number, n_number)), np.zeros((a_number, n_number))
for i in range (0, a_number):
    for j in range (0, n_number):
        mx_m[i,j], fx_m[i,j], w_m[i,j], p_m[i,j] = mx[k], fx[k], w[k], p[k]
        k += 1    

# Построение графиков

## Общие графики

### Fx

In [6]:
Fx = graph(go.Figure(), fx_m, fx, df, a_number, n_number, 'Fx, Н')

f_1 = 35
n_cross_1 = cross_points_f(f_1, a_number, n_number, fx_m, df)
Fx.cross_line(n_cross_1,  [f_1]*len(n_cross_1), f_1)
Fx.basic_line(n_cross_1, list(range(0, round(max(df['a, град']))+1, 1)), f_1)
# Fx.annotation(n_cross_1, list(range(0, round(max(df['a, град']))+1, 1)), f_1)
Fx.horizontal_cross_palne(f_1)

f_2 = 20
n_cross_2 = cross_points_f(f_2, a_number, n_number, fx_m, df)
Fx.cross_line(n_cross_2,  [f_2]*len(n_cross_1), f_2, 'white')
Fx.horizontal_cross_palne(f_2)

Fx.basic_plot([[0,'rgb(255, 0, 0)'], [1, 'rgb(255, 0, 0)']])

/home/burger/Documents/works/bpla/func/cross_points_f.py:26: RuntimeWarning:

invalid value encountered in divide



### Mx

In [7]:
Mx = graph(go.Figure(), mx_m, mx, df, a_number, n_number, 'mx, Н*м')

Mx.cross_line(n_cross_1,  cross_points_mx_w(n_cross_1, a_number, n_number, mx_m, df), f_1)

Mx.cross_line(n_cross_2,  cross_points_mx_w(n_cross_2, a_number, n_number, mx_m, df), f_2, 'white')

Mx.basic_plot([[0,'rgb(255, 165, 0)'], [1, 'rgb(255, 165, 0)']])

### W

In [8]:
W = graph(go.Figure(), w_m, w, df, a_number, n_number, 'Wв, Вт')

W.cross_line(n_cross_1,  cross_points_mx_w(n_cross_1, a_number, n_number, w_m, df), f_1)
W.cross_line(n_cross_2,  cross_points_mx_w(n_cross_2, a_number, n_number, w_m, df), f_2, 'white')

W.basic_plot([[0,'rgb(66, 170, 255)'], [1, 'rgb(66, 170, 255)']])

### P

In [9]:
P = graph(go.Figure(), p_m, p, df, a_number, n_number, 'P, Н/Вт')

P.cross_line(n_cross_1,  cross_points_mx_w(n_cross_1, a_number, n_number, p_m, df), f_1)
P.cross_line(n_cross_2,  cross_points_mx_w(n_cross_2, a_number, n_number, p_m, df), f_2, 'white')

P.basic_plot([[0,'rgb(144, 238, 144)'], [1, 'rgb(144, 238, 144)']])